# Unsupervised agent demonstration

Here we demonstrate the use of a filesystem based `DirectoryAgent,`
using the `NMFCompanion` agent that follows the `tell`--`report`--`ask` paradigm.

In [ ]:
from bnl_ml.unsupervised.agent import NMFCompanion
from bnl_ml.utils.filesystem import ObservationalDirectoryAgent
from pathlib import Path
import numpy as np

First we set up a linspace that contains the ROI for the measurement,
and a data transformation that takes the complete measurement (Q, I(Q))
and trims it to the region on interest.

In [ ]:
x_linspace = np.linspace(0, 10, 545)


def data_transform(data):
    """Trim data to Region of interest"""
    x, y = data
    idx_min = (
        np.where(x < x_linspace[0])[0][-1] if len(np.where(x < x_linspace[0])[0]) else 0
    )
    idx_max = (
        np.where(x > x_linspace[-1])[0][0]
        if len(np.where(x > x_linspace[-1])[0])
        else len(y)
    )
    return x[idx_min:idx_max], y[idx_min:idx_max]

Next we instatiate the NMF companion. This agent takes care of
the data normalization, the model construction using the scikit-learn backend,
and the instructions for what to do when told about data, or asked for a report.

In this case, report constructs a plot that encompases the NMF decomposition
related to the independent variable of the experiment (temperature).

In [ ]:
companion = NMFCompanion(4, x_linspace=x_linspace)

Following this, we construct the `DirectoryAgent.`
This takes the place of a Bluesky callback or service, when dealing with
local file streams instead of document streams.
It has some extra options to draw the experimental independent variable
out of a path manipulation, as users will commonly store essential metadata in filenames.

#### This agent will pay attention to a tmp directory in the example_data folder.

In [ ]:
directory = Path("../") / "example_data" / "tmp"
directory.mkdir(exist_ok=True)

da = ObservationalDirectoryAgent(
    companion,
    directory,
    path_spec="*.chi",
    data_transform=data_transform,
    independent_from_path=lambda path: float(path.name.split("_")[-2][1:-1]),
)
da.load_dir()

Finally, we spin the agent. This is a continuous operation to
watch and react to the target directory (`example_data/tmp`).
After executing the following cell, go ahead and copy some or all of the data from
`example_data/NaCl_CrCl3_pdf_ramp` into `example_data/tmp`.

The spin will only run for 2 minutes, before stopping.

In [ ]:
da.spin(sleep_delay=5, timeout=120)